In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'): #datos de la competencia
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as mticker
from PIL import Image
import matplotlib.dates as mdates
import matplotlib.image as img

#Model Libraries
import gc
from sklearn.metrics import mean_squared_log_error

#
import warnings
warnings.filterwarnings('ignore')

## Cargar Datos 

In [ ]:
DATA_PATH = '../input/store-sales-time-series-forecasting/'

In [ ]:
for f in os.listdir(DATA_PATH):
    print(f)

In [ ]:
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_stores = pd.read_csv(os.path.join(DATA_PATH, 'stores.csv'))
df_transactions = pd.read_csv(os.path.join(DATA_PATH, 'transactions.csv'))
df_oil = pd.read_csv(os.path.join(DATA_PATH, 'oil.csv'))
#df_holidays = pd.read_csv(os.path.join(DATA_PATH, 'holidays_events.csv'))

In [ ]:
df_train2 = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'), 
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={ # set datatypes in advance for ease of use
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,)

df_holidays = pd.read_csv(
    DATA_PATH + 'holidays_events.csv',
    dtype={ # set datatypes in advance for ease of use
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)

df_holidays = df_holidays.set_index('date').to_period('D')

df_train2['date'] = df_train2.date.dt.to_period('D')
store_sales = df_train2.set_index(['store_nbr', 'family', 'date']).sort_index()

In [ ]:
df_train.describe()

In [ ]:
df_train.head()

In [ ]:
# Convertir a formato de tipo fecha en lugar de string y se guarda en la misma columna
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test.date) # Misma format de hacer lo anterior. Hace referencia al título de la columna
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

In [ ]:
df_train.head()

In [ ]:
# Cambiar formato de datos
df_train['onpromotion'] = df_train.onpromotion.astype('float16')
df_train['sales'] = df_train.sales.astype('float32')
df_stores.cluster = df_stores.cluster.astype('int8')

## Análisis de Exploración de los Datos

In [ ]:
df_train

In [ ]:
average_sales = df_train.groupby('date').mean()['sales']

In [ ]:
average_sales

In [ ]:
# Copia de la variable
avg_train = average_sales.copy()

In [ ]:
# Calcular promedio de un año, luego el siguiente, etc. Promedio por años
moving_avg = avg_train.rolling(window=365, center=True, min_periods=183).mean() #183 porque es la mitad del 365

In [ ]:
avg_train.plot(style='.', figsize=(20,10))
plt.ylabel('Ventas')
plt.title('Ventas promedio por año')
moving_avg.plot()

In [ ]:
average_transactions = df_transactions.groupby('date').mean()['transactions']
avg_trans = average_transactions.copy()
moving_avg_trans = avg_trans.rolling(window=365, center=True, min_periods=183).mean()

In [ ]:
avg_trans.plot(style='.', figsize=(20,10))
plt.ylabel('Transacciones')
plt.title('Transacciones promedio por año')
moving_avg_trans.plot()

In [ ]:
#Agrupar por fecha y tienda. Suma las ventas por tiena y por fecha y lo vamos a utilizar como índice
df_train_aux = df_train.groupby(['date', 'store_nbr']).sales.sum().reset_index() 

In [ ]:
# Combinar el train_aux con el transactions
df_aux_merged = pd.merge(df_train_aux, df_transactions, how='left')

In [ ]:
df_aux_merged

In [ ]:
pearson_corr = df_aux_merged.corr('pearson')['sales'].loc['transactions']
spearman_corr = df_aux_merged.corr('spearman')['sales'].loc['transactions']

Gracias a las correlaciones veo proporcionalidades. Ver cuáles son las variables o atributos de un data set que tiene mayor relación, dependencias lineales

In [ ]:
# atributos de un data set que tiene mayor relación, dependencias lineales
print(pearson_corr, spearman_corr)

In [ ]:
df_sorted_trans = df_transactions.sort_values(['store_nbr', 'date'])

Gráfica de transacciones realizadas por cada tienda a lo largo de las fechas analizadas  


In [ ]:
# Cada color es una tienda diferente
px.line(df_sorted_trans, x='date', y='transactions', title='Transacciones por día', color='store_nbr')

In [ ]:
aux_a = df_transactions.copy()
aux_a['year'] = aux_a.date.dt.year # Sacar el año. TOmar fecha, tomar como tipo datetime (dt) y tomar el año
aux_a['month'] = aux_a['date'].dt.month

In [ ]:
aux_a

Gráfica de caja de  las transacciones por cada mes en los diferentes años, este tipo de gráficas nos ayudan a identificar los valores extremos e identificar cómo se comportan las transacciones cada mes 

In [ ]:
px.box(aux_a, x='year', y='transactions', title='Transacciones por mes', color='month')

In [ ]:
# Promedio por mes
aux_a = df_transactions.set_index('date').resample('M').transactions.mean().reset_index()

In [ ]:
aux_a['year'] = aux_a.date.dt.year

Gráfica de las transacciones por año, podemos observar como las transacciones suben a final de año , de manera significativa, más adelante veremos como es el comportamiento de las ventas en esos periodos de tiempo para identificar si existe alguna relación

In [ ]:
px.line(aux_a, x='date', y='transactions', title='Transacciones por año', color='year')

Gráfica  para mostrar la relación entre las ventas y las transacciones realizadas, como podemos ver con la tendencia lineal positiva, estas presentan una relación positiva es decir cuando una aumenta la otra igual aumenta 

In [ ]:
# Correlación positiva
px.scatter(df_aux_merged, x='transactions', y='sales', trendline='ols', title='Transacciones contra ventas', trendline_color_override = 'red')

In [ ]:
aux_a = df_transactions.copy()
# Crear columnas de año y día de la semanaGráfica de las ventas realizadas por cada tienda a lo largo del tiempo, podemos observar como tenemos valores en la parte de abajo cuyas ventas fueron de cero los primeros meses.Una vez que observamos las fechas donde dejan de tener días seguidos con ventas de 0 asumimos que es cuando se abren las tiendas y tomamos la misma fecha pero con un año más para utilizarla como referencia de punto de partida de los datos que son significativos para nuestro modelo, ya que si la tienda acaba de abrir el primer año sus ventas no serán muchas y puede afectar el modelo. Con este punto de partida eliminamos las filas de dichas tiendas donde la fecha sea menor a la propuesta,
aux_a['year'] = aux_a['date'].dt.year
aux_a['dayofweek'] = aux_a['date'].dt.dayofweek + 1 #1 para que empiece en lunes
aux_a = aux_a.groupby(['year', 'dayofweek'])

In [ ]:
aux_a.head(10)

In [ ]:
aux = df_train.set_index('date').groupby('store_nbr').resample('D').sales.sum().reset_index()

Gráfica de las ventas realizadas por cada tienda a lo largo del tiempo.
Al graficar las ventas totales diarias de cada una de las tiendas observamos un aspecto importante, que es que en la parte inferior tenemos tiendas cuyas ventas son de 0 hasta cierto periodo de tiempo, esto se debe a que algunas tiendas abrieron mucho después pero dentro del dataset no está indicado simplemente marcan las ventas como 0. En este caso estos registros no aportan ninguna información necesaria para nuestro modelo, al contrario podría afectar la predicción ya que asumirá que durante un periodo de tiempo nuestras ventas fueron menores dado que tenemos valores de 0. Por lo tanto podemos identificar cuantos registros son los que se encuentran en ventas iguales a 0 para cada tienda.
Una vez que observamos las fechas donde dejan de tener días seguidos con ventas de 0 asumimos que es cuando se abren las tiendas y tomamos la misma fecha pero con un año más para utilizarla como referencia de punto de partida de los datos que son significativos para nuestro modelo, ya que si la tienda acaba de abrir el primer año sus ventas no serán muchas y puede afectar el modelo. Con este punto de partida eliminamos las filas de dichas tiendas donde la fecha sea menor a la propuesta.

In [ ]:
px.line(aux, x='date', y='sales',title='Ventas por día', color='store_nbr')

In [ ]:
df_train[((df_train.store_nbr == 52) & (df_train.date < '2017-04-20'))]

In [ ]:
print(df_train.shape)

In [ ]:
# Reescribir el dataframe
df_train = df_train[~((df_train.store_nbr == 52) & (df_train.date < '2017-04-20'))]
df_train = df_train[~((df_train.store_nbr == 22) & (df_train.date < '2015-10-09'))]
df_train = df_train[~((df_train.store_nbr == 42) & (df_train.date < '2015-08-21'))]
df_train = df_train[~((df_train.store_nbr == 21) & (df_train.date < '2015-07-24'))]
df_train = df_train[~((df_train.store_nbr == 29) & (df_train.date < '2015-03-20'))]
df_train = df_train[~((df_train.store_nbr == 20) & (df_train.date < '2015-02-13'))]
df_train = df_train[~((df_train.store_nbr == 53) & (df_train.date < '2014-05-29'))]
df_train = df_train[~((df_train.store_nbr == 36) & (df_train.date < '2013-05-09'))]

In [ ]:
df_train.shape

In [ ]:
# Crea una nueva columna llamada ind que va a ser un acumulador
aux = df_train[['store_nbr', 'sales']].copy()
aux['ind'] = 1
# Suma en la columna int / Acumula el número de registros por tienda / Contar el número de registros por tienda
aux['ind'] = aux.groupby('store_nbr').ind.cumsum().values

In [ ]:
aux

In [ ]:
aux = pd.pivot(aux, index='ind', columns='store_nbr', values='sales')

In [ ]:
aux

In [ ]:
# Correlación
aux = aux.corr()

In [ ]:
# Coorrelación entre tiendas
aux

In [ ]:
plt.figure(figsize=(20,20))
mask = np.triu(aux)
sns.heatmap(aux,
           annot=True,
           fmt='.1f',
           cmap='coolwarm',
           square='True',
           linewidths=1,
           cbar=False,
           mask=mask)
plt.show()

# ¿El Terremoto del 2016 afectaron las ventas?

Dentro de la descripción de datos se nos menciona que: “Un terremoto de magnitud 7,8 sacudió Ecuador el 16 de abril de 2016. La gente se unió a los esfuerzos de ayuda donando agua y otros productos de primera necesidad, lo que afectó en gran medida las ventas de los supermercados durante varias semanas después del terremoto.”

Primero vamos a analizar el promedio de ventas que se tuvo en cada año en el mes de Abril a Mayo que es cuando sucedió el terremoto.


In [ ]:
df_ter= pd.merge(df_train,df_stores)
df_ter['month'] = df_ter.date.dt.month.astype("int8")
df_ter['day_of_month'] = df_ter.date.dt.day.astype("int8")
df_ter['day_of_year'] = df_ter.date.dt.dayofyear.astype("int16")
df_ter['week_of_month'] = (df_ter.date.apply(lambda d: (d.day-1) // 7 + 1)).astype("int8")
df_ter['week_of_year'] = (df_ter.date.dt.weekofyear).astype("int8")
df_ter['day_of_week'] = (df_ter.date.dt.dayofweek + 1).astype("int8")
df_ter['year'] = df_ter.date.dt.year.astype("int32")
df_ter[(df_ter.month.isin([4,5]))].groupby(["year"]).sales.mean()

En este punto podemos ver que si bien hubo una diferencia notable entre el año 2015 al 2016, una vez que graficamos la distribución de las ventas de todos los meses por año (gráfica que se muestra a continuación) podemos observar que el crecimiento en ventas por mes era igual para los demás meses.

In [ ]:
average_sales = (
    df_train
    .groupby('date').mean()
    .squeeze()
    #.loc['2016']
)
moving_avg=average_sales['sales'].rolling(window=365,center=True,min_periods=183).mean()
average_sales['sales'].plot(style='.')
plt.ylabel('Ventas')
plt.title('Ventas promedio por año')
moving_avg.plot()

Hasta este punto podemos descartar que se tuvo un impacto relevante para las ventas generales de ese mes comparado a los anteriores años e incluso a los anteriores meses ya que la tendencia de las ventas parece ser lineal positiva.
Sin embargo uno de los puntos que tenemos que tomar en cuenta es que no solo se analizan las ventas generales sino las ventas por familia de productos, para identificar si se tuvo o no una diferencia notable en las ventas de cada familia de productos en ese mes los compararemos con el promedio de venta de el mes anterior y el mes siguiente


In [ ]:
mar_data=pd.pivot_table(df_ter[(df_ter.month.isin([3]))], index="year", columns="family", values="sales", aggfunc="mean")
mar_data
px.line(mar_data)

In [ ]:
abr_may_data=pd.pivot_table(df_ter[(df_ter.month.isin([4,5]))], index="year", columns="family", values="sales", aggfunc="mean")
abr_may_data
px.line(abr_may_data)

In [ ]:
jun_data=pd.pivot_table(df_ter[(df_ter.month.isin([6]))], index="year", columns="family", values="sales", aggfunc="mean")
jun_data
px.line(jun_data)

Con las gráficas podemos observar que si bien se tuvo un pequeño aumento en el consumo de cierta familia de productos, la diferencia con los demás meses y años no es lo suficientemente extrema como para afectar el desarrollo de nuestro modelo por lo que no es necesario limpiar estos datos. 

# ¿Las festividades(holidays) afectan las ventas?

In [ ]:
patternDel = "Terremoto.*" # Filter out the earthquake of 2016
filter1 = df_holidays['description'].str.contains(patternDel)
df_holidays = df_holidays[~filter1]
df_holidays

Se buscó analizar si dichos eventos tienen algún efecto en las ventas, para realizar esto primero se condensaron todas las ventas por tienda, familia de producto y fecha, también se removió los datos del terremoto de 2016 de la tabla de holidays, debido a que fue un evento extraordinario.

Seguidamente se condensaron las ventas diarias para los años 2013, 2014, 2015 y 2016, debido a que estos son los años de los que tenemos información completa. Para la tabla de holidays nos enfocamos en los eventos que fueron a nivel nacional y regional, esto debido a que consideramos los que pueden tener una mayor efecto en el número total de ventas.

Al graficar los datos, se pueden observar las siguientes situaciones:


In [ ]:
l = ['2013','2014','2015','2016']
#l = ['2013','2014']

for i in l: # For loop that creates a mean sales by year and a scatter of the holidays of said year
    average_sales = ( # group all store sales by date, get the mean of all sales in a date & condence df to just date and sale #
        store_sales
        .groupby('date').mean()
        .squeeze()
        .loc[i]
    )
    holidays = ( # Get all National and Regional holidays in a given year
    df_holidays
    .query("locale in ['National', 'Regional']")
    .loc[i, ['description']]
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
    )
    
    # Convert df's index from date to st
    average_sales.index =average_sales.index.astype(str)
    holidays.index =holidays.index.astype(str)

    # Remove december 25 from holidays df, store does not open that day
    holidays = holidays.drop(labels = [(i + '-12-25')]) 
    
    t='year ' + i
    print(i)
    
    plt.figure(figsize=(12,6)) # Set figsize
    plt.ylabel('Sales') # Set ylabel

    # displaying the title
    plt.title("Average sales and holidays, " + t)
    plt.plot(average_sales.index,average_sales.values) # plot average sales for date
    plt.scatter(holidays.index, average_sales[holidays.index],color='C3') # scatter plot of holidays
    plt.grid() # add grid
    
    #plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.xticks(rotation = 90)
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(bymonthday=[1,16]))# Locate days 1 and 16 of all months
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))# put tick in days 1 & 16 of all months, format is day-month
    
    #plt.savefig(t+'.png', dpi = 300) #save graph as year n.png
    #plt.clf() # Clear graph, plt has troubles rendering the differents plots

Podemos observar que hay un incremento en el número de ventas diarias alrededor de las fechas en las que sucede un evento, de estos es particularmente interesante que los primeros días del mes de enero se tiene un número de ventas muy bajo, esto se puede asociar al incremento de ventas que tomó lugar en los últimos días del mes de diciembre del año pasado; también está el incremento en ventas alrededor de los días de la madre, el viernes santo, el grito de la independencia, el dia de difuntos y los días de ofertas de Black Friday y Cyber Monday.

Con la ayuda de estas visualizaciones podemos decir que en el periodo de tiempo alrededor de días donde ocurren eventos se pueden observar un aumento en el número de ventas.


In [ ]:
# display image, for a better resolution directly open the file
#2013
#imge = img.imread('../working/year 2013.png')
#imgplot = plt.imshow(imge)

In [ ]:
holidays = (df_holidays
    .query("locale in ['National', 'Regional']")
    .loc['2013', ['description']]
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
    )
holidays

# Transformamos las festividades a formato onehot

como observamos en la parte de arriba estas influyen en las ventas de nuestros productos por lo que son datos importantes paara nuestro modelo , pero estas deben de transformarse ya que actualmente solo son strings, el formato onehot nos permite crear un array marcando 1 en la festividad que se tiene y 0 si no es la correcta.

In [ ]:
holiday_temp = df_holidays.copy()

mask = ~holiday_temp.loc[:,'description'].str.contains(r'Terremoto.*', case = False)
holiday_temp = holiday_temp.loc[mask].copy()



holiday_temp = (df_holidays
    .query("locale in ['National', 'Regional']")
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
    )
holiday_temp = holiday_temp[["description"]].copy()
#year_l = ['2013','2014','2015','2016','2017']
#for i in year_l:
#holiday_temp2 = holiday_temp1.drop(labels = [('2017-12-25')]) 
#holiday_temp1 = holiday_temp1.drop(holiday_temp1[holiday_temp1['description'] == 'Navidad'].index)
hols = ['Navidad','Black Friday','Cyber Monday',
        'Inauguracion Mundial de futbol Brasil',
        'Mundial de futbol Brasil: Ecuador-Suiza',
        'Mundial de futbol Brasil: Ecuador-Honduras',
        'Mundial de futbol Brasil: Ecuador-Francia',
        'Mundial de futbol Brasil: Octavos de Final',
        'Mundial de futbol Brasil: Semifinales',
        'Mundial de futbol Brasil: Tercer y cuarto lugar',
        'Mundial de futbol Brasil: Cuartos de Final',
        'Mundial de futbol Brasil: Final']

holiday_temp1 = holiday_temp[holiday_temp.description.isin(hols) == False].copy()
holiday_temp1

In [ ]:
list(holiday_temp1.columns)

In [ ]:
list(holiday_temp1['description'].unique())

In [ ]:
#holiday_1hot = df_holidays2['description']
holiday_1hot = pd.get_dummies(holiday_temp1, dtype=int, prefix='', prefix_sep='')
holiday_1hot= holiday_1hot[['Provincializacion de Cotopaxi','Provincializacion de Imbabura',
 'Primer Grito de Independencia',
 'Independencia de Guayaquil',
 'Dia de Difuntos',
 'Independencia de Cuenca',
 'Provincializacion de Santo Domingo',
 'Provincializacion Santa Elena',
 'Navidad-4',
 'Navidad-3',
 'Navidad-2',
 'Navidad-1',
 'Navidad+1',
 'Primer dia del ano',
 'Carnaval',
 'Viernes Santo',
 'Dia del Trabajo',
 'Dia de la Madre-1',
 'Dia de la Madre',
 'Batalla de Pichincha']]
holiday_1hot

In [ ]:
list(holiday_1hot.columns)

# Analizar el total de ventas por familia de productos en cada tienda

In [ ]:
store_family = df_train.groupby(["family", "store_nbr"]).tail(60).groupby(["family", "store_nbr"]).sales.sum().reset_index()
store_family

Por otra parte es importante revisar que no tengamos ninguna anormalidad con respecto a nuestras ventas por familia de productos en las tiendas, para eso graficamos la suma total de las ventas para cada una de las tiendas respecto a la familia de productos existentes, obteniendo la siguiente gráfica:


In [ ]:
px.line(store_family, x = "store_nbr", y= "sales", color = "family", title = "Ventas totales de la familia en cada tienda")

Como podemos observar en la parte inferior de la gráfica, existen productos cuya sumatoria total de venta de cierta familia de productos es igual a 0, esto es debido a que varias tiendas no ofrecen todos los productos por lo que la predicción de la venta de estos siempre será 0, por lo que no es necesario generar una predicción para estos caso y podemos omitirla de nuestro dataset, sin olvidar que al momento  de generar nuestras predicciones tenemos que tomar en cuenta que si las ventas que queremos predecir de una tienda con una familia de productos que se encuentra entre estos nuevos valores su predicción será 0.

In [ ]:
store_family[store_family.sales == 0]

Como podemos ver en efecto hay tiendas que no manejan la venta de algunos productos por lo tanto no son necesarias para nuestro modelo de predicción ya que dichas tiendas para ese producto su predicción simpre sera de cero ventas

In [ ]:
#Retiramos esos valores de nuestro DataFrame
print(df_train.shape)
# Anti Join
outer_join = df_train.merge(store_family[store_family.sales == 0].drop("sales",axis = 1), how = 'outer', indicator = True)
df_train = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
del outer_join
gc.collect()
df_train.shape

# Analizar si el precio del petróleo influye en las ventas

Al ver la descripción de este reto se nos dice que Ecuador es un país dependiente del petróleo y puede ser vulnerable a los cambios del precio de este. Es por esto que se quiere averiguar si de verdad el precio del petróleo representa un parámetro considerable que podría afectar las ventas.Al realizar una gráfica de dispersión comparando las ventas por día con el cambio de precio del petróleo por día, se puede ver que estos no tienen una relación tan estrecha como para considerar al precio del petróleo como algo que repercute de manera importante en las ventas, tampoco existe una aparente correlación, por lo que no será tomado en cuenta.

In [ ]:
new = df_train.groupby(["date", "store_nbr"]).sales.sum().reset_index()
new["date"] = pd.to_datetime(new.date)
new

In [ ]:
df_oil["date"] = pd.to_datetime(df_oil.date)
# Resample
df_oil = df_oil.set_index("date").dcoilwtico.resample("D").sum().reset_index()
# Interpolate
df_oil["dcoilwtico"] = np.where(df_oil["dcoilwtico"] == 0, np.nan, df_oil["dcoilwtico"])
df_oil["dcoilwtico_interpolated"] = df_oil.dcoilwtico.interpolate()
# Plot
p = df_oil.melt(id_vars=['date']+list(df_oil.keys()[5:]), var_name='Legend')
px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend',title = "Precio diario del petróleo" )

In [ ]:
temp = pd.merge(new, df_oil, how = "left")
temp

In [ ]:
print("Correlation con el precio del petróleo")
print(temp.drop(["store_nbr", "dcoilwtico"], axis = 1).corr("spearman").dcoilwtico_interpolated.loc[["sales"]], "\n")

Al realizar una gráfica de dispersión comparando las ventas por día con el cambio de precio del petróleo por día, se puede ver que estos no tienen una relación tan estrecha como para considerar al precio del petróleo como algo que repercute de manera importante en las ventas, tampoco existe una aparente correlación, por lo que no será tomado en cuenta.

In [ ]:
temp.plot.scatter(x = "dcoilwtico_interpolated", y = "sales", title="Precio del petróleo contra ventas")

#  Formato One hot  para las familias

como observamos en la parte de arriba y en el objetivo del reto, queremos predecir las ventas por familia y tienda por lo que esta información se debe incluir para el modelo, pero estas deben de transformarse ya que actualmente solo son strings, el formato onehot nos permite crear un array marcando 1 en la festividad que se tiene y 0 si no es la correcta.


In [ ]:
df_train

In [ ]:
family_1hot = pd.get_dummies(df_train.family, dtype=int)
family_1hot

In [ ]:
df_train_new = df_train.drop(["family"], axis=1)
df_train_new

In [ ]:
df_train_new = df_train_new.join(family_1hot)

# Merge con holidays

Juntamos la tabla de holidays_1hot con nuestros datos de train para poder entrenar el modelo con los datos completos 

In [ ]:
holiday_1hot

In [ ]:
holiday_1hot = holiday_1hot.reset_index()
holiday_1hot['date'] = holiday_1hot['date'].astype('datetime64[ns]') # convert holiday_1hot['date'] to the type of df_train_new['date']


In [ ]:
holiday_1hot

In [ ]:
df_train_merged = pd.merge(df_train_new, holiday_1hot, how='left')
df_train_merged = df_train_merged.fillna(0) # replace all NaN to 0
df_train_merged

# Crear Test

Adaptamos la tabla de test incluyendo la información de holiday y family con el formato One Hot

In [ ]:
df_test

In [ ]:
testfamily_1hot = pd.get_dummies(df_test.family, dtype=int)
testfamily_1hot

In [ ]:
df_test_new = df_test.drop(["family"], axis=1)
df_test_new

In [ ]:
df_test_new = df_test_new.join(testfamily_1hot)

In [ ]:
df_test_new

In [ ]:
holidays_t = (df_holidays
    .query("locale in ['National', 'Regional']")
    .loc['2017', ['description']]
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
    ).copy()
holidays_t = holidays_t.drop(labels = [('2017-12-25')]) 
holidays_t

In [ ]:
holiday_1hot_t = pd.get_dummies(holidays_t, dtype=int, prefix='', prefix_sep='')
holiday_1hot_t = holiday_1hot_t.reset_index()
holiday_1hot_t['date'] = holiday_1hot_t['date'].astype('datetime64[ns]') # convert holiday_1hot['date'] to the type of df_train_new['date']
holiday_1hot_t = holiday_1hot_t[['date','Provincializacion de Cotopaxi','Provincializacion de Imbabura',
 'Primer Grito de Independencia',
 'Independencia de Guayaquil',
 'Dia de Difuntos',
 'Independencia de Cuenca',
 'Provincializacion de Santo Domingo',
 'Provincializacion Santa Elena',
 'Navidad-4',
 'Navidad-3',
 'Navidad-2',
 'Navidad-1',
 'Navidad+1',
 'Primer dia del ano',
 'Carnaval',
 'Viernes Santo',
 'Dia del Trabajo',
 'Dia de la Madre-1',
 'Dia de la Madre',
 'Batalla de Pichincha']]

holiday_1hot_t

In [ ]:
df_test_merged = pd.merge(df_test_new, holiday_1hot_t, how='left')
df_test_merged = df_test_merged.fillna(0) # replace all NaN to 0
#df_test_merged = df_test_merged.drop(["date"], axis=1)
df_test_merged

In [ ]:
list(df_train_merged.columns)

In [ ]:
list(df_test_merged.columns)

In [ ]:
def modelRMSE(yTrue, yPred):
    return mean_squared_log_error(yTrue, yPred,squared=False)
    

# Regresión lineal

Para este modelo utilizamos una regresión lineal simple, utilizando todas las columnas , pero para evitar tener datos muy extremos aplicamos winsorización de 10% para que los valores extremos se pasen a los valores del 10 percentil 

In [ ]:
#Winsorizacion 
from scipy.stats.mstats import winsorize
df_train_merged['sales']=winsorize(df_train_merged['sales'],(0.1,0.1))
x_train = df_train_merged.drop(['sales'], axis=1)
y_train = df_train_merged['sales']

In [ ]:
x_train

In [ ]:
x_train_sinDate = x_train.set_index('date')

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
model

In [ ]:
model.fit(x_train_sinDate,y_train)

In [ ]:
model.coef_

In [ ]:
df_test_new
x_test = df_test_merged.set_index('date')

In [ ]:
model.predict(x_test)

In [ ]:
#y_submit = pd.DataFrame(model.predict(x_test), index=x_test.id, columns=['sales'])
#y_submit = y_submit.stack(['store_nbr', 'family'])
#y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
#y_submit = y_submit.reset_index()
#y_submit.to_csv('submission.csv', index=False)

Los resultados de este modelo no fueron optimos ya que el modelo era muy simple para la cantidad de datos y coeficientes a predecir.

# Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
"""params = {
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

grid_search = GridSearchCV(RandomForestRegressor(), #GridSearchCV will use brute force to find the best 
                           param_grid= params,      #combination of the given parameters with the best error
                           scoring= 'neg_root_mean_squared_error',
                           refit=True,
                           cv= RepeatedKFold(n_splits=5, n_repeats=3),
                           n_jobs=-1)"""
grid_search=RandomForestRegressor(n_estimators=5) #Best parameters found was n_estimators = 5, the rest are default
grid_search.fit(x_train_sinDate,y_train)
#print(grid_search.best_estimator_)

In [ ]:
y_preds = grid_search.predict(x_test) #Predict the test data
y_preds

In [ ]:
y_pred2=grid_search.predict(x_train_sinDate) #Predict the RMSE with the training data
modelRMSE(y_train, y_pred2)

In [ ]:
y_sub = pd.DataFrame(y_preds, index=x_test.id, columns=['sales']) #Modify the data to match the required format in the competition
y_sub = y_sub.reset_index()
y_sub

In [ ]:
y_sub.to_csv('submission.csv', index=False)

# XGB

In [ ]:
import xgboost as xgb

In [ ]:
"""xgb_train_x = xgb.DMatrix(data=x_train_sinDate, label=y_train)# Modify dataset into an custom XGB data structure for speed
xgb_train_x"""

In [ ]:
"""xgb_train_x2 = xgb.DMatrix(data=x_train_sinDate) # Modify dataset into an custom XGB data structure for speed
xgb_train_x2"""

In [ ]:
"""xgb_test = xgb.DMatrix(data=x_test) # Modify dataset into an custom XGB data structure for speed
xgb_test"""

In [ ]:
"""params = {"booster":"gbtree", "objective":"reg:linear","eval_metric":"rmsle"}""" #Params

In [ ]:
"""xgb_r = xgb.train(params=params, dtrain = xgb_train_x, num_boost_round = 10)""" #train model using given params

In [ ]:
"""pred = xgb_r.predict(xgb_test)#Predict test data
pred"""

In [ ]:
"""from sklearn.preprocessing import minmax_scale
pred2 = xgb_r.predict(xgb_train_x2)
pred2=minmax_scale(pred2, feature_range=(0,1))#Convert data to remove negatives
modelRMSE(y_train, pred2)"""

In [ ]:
"""y_sub = pd.DataFrame(pred, index=x_test.id, columns=['sales'])
y_sub = y_sub.reset_index()
y_sub"""

In [ ]:
#y_sub.to_csv('submission.csv', index=False)

# Seasonality

In [ ]:
#importamos las librerias necesarias 
"""
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_periodogram, seasonal_plot
"""

In [ ]:
# Importamos nuevamente train y ajustamos los tipos de  variables 

"""
from pathlib import Path
comp_dir = Path('../input/store-sales-time-series-forecasting')
store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()"""


In [ ]:
# Agrupamos las ventas para sacar la media
"""average_sales = (
    store_sales
    .groupby('date').mean()
    .squeeze()
)"""

Transformación de Fourier: La transformada de Fourier permite transformar una función de tiempo y señal en una función de frecuencia y potencia. Esto le indica qué frecuencias componen su señal y qué tan fuertes son. 

In [ ]:
#Agregamos los futures con transformada de Fourier
"""from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
y= average_sales.copy()
fourier = CalendarFourier(freq='A', order=100)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()"""

In [ ]:
#Corremos el modelo con los datos y hacemos las predicciones 
"""model = LinearRegression().fit(X, y)
y_pred = pd.Series(
    model.predict(X),
    index=X.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X), index=X.index)
ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax.legend();"""

In [ ]:
#Ahora agregamos las columnas de holidays con one hot 
"""
X_holidays = pd.get_dummies(holidays.drop_duplicates())

X2 = X.join(X_holidays, on='date').fillna(0.0)"""

In [ ]:
"""X2.columns.values"""

In [ ]:
#Corremos el modelo nuevamente psrs ver si este se ajusta mejor 
"""model = LinearRegression().fit(X2, y)
y_pred = pd.Series(
    model.predict(X2),
    index=X2.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X2), index=X2.index)
ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax.legend();"""

In [ ]:
#Cargamos los datos de test y agregamos los features de seasonality y holidays 
#Al finalizar de predecir el modelo , guardamos los datos en formato csv.
"""df_test = pd.read_csv(
    comp_dir / 'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

# Create features for test set
X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'


X_holidays = pd.get_dummies(holidays)
X2_test = X_test.join(X_holidays, on='date').fillna(0.0)


y_submit = pd.DataFrame(model.predict(X2_test), index=X2_test.index, columns=['sales'])

y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit = y_submit.reset_index()
y_submit=y_submit.drop(['store_nbr', 'family','date'], axis=1)
y_submit.to_csv('submission.csv', index=False)"""

In [ ]:
"""X_test"""